## Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.tools as tls
from ipywidgets import widgets
import datetime
import warnings
warnings.filterwarnings("ignore")

/Users/atroboukis/.pyenv/versions/3.8.1/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

## Data URLs 

In [27]:
baseHopkinsURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"
confirmedHopkinsFileName='time_series_covid19_confirmed_global.csv'
deathsHopkinsFileName='time_series_covid19_deaths_global.csv'

# τα σκραπαρισμένα data από worldometer
worldOmeterData='https://app.workbenchdata.com/public/moduledata/live/308692.csv'

# από το αρχείο countries_names στο google spreadsheet
countriesInGreek='https://docs.google.com/spreadsheets/d/e/2PACX-1vRpR8AOJaRsB5by7H3R_GijtaY06J8srELipebO5B0jYEg9pKugT3C6Rk2RSQ5eyerQl7LolshamK27/pub?gid=906157277&single=true&output=csv'

# population
population='https://app.workbenchdata.com/public/moduledata/live/302569.csv'

# Line Chart - logarithmic - after XX cases | XX deaths

In [60]:
'''φορτώνουμε το αρχείο που επιθυμούμε, τον αριθμό σημείο έναρξης της οπτικοποίησης,
το όνομα της νέας στήλης που θα δημιουργηθεί από το melt, 
το όνομα του αρχείου που θέλουμε να σώσουμε'''

def after100Cases(fileName, numberCompare, columnName, outputName, titleGraphic):
    
    ''' φορτώνουμε το πρώτο αρχείο από το Hopkins'''
    df=pd.read_csv(baseHopkinsURL + fileName)\
        .drop(['Lat', 'Long'], axis=1) \
        .melt(id_vars=['Province/State', 'Country/Region'], 
                 var_name='Date', value_name=columnName) \
        .astype({'Date':'datetime64[ns]', columnName:'Int64'}, 
                 errors='ignore')
    
    ''' φορτώνουμε το αρχείο με την αντιστοίχιση ελληνικών - αγγλικών ονομάτων χωρών'''
    gr=pd.read_csv(countriesInGreek)
    ''' το συνδέουμε με το dataframe από το Hopkins ώστε να προστεθεί το ADMIN_GR'''
    df=pd.merge(df,gr, how='left', left_on='Country/Region', right_on='ADMIN')
    
    ''' φορτώνουμε το αρχείο με τους πληθυσμούς'''
    pop=pd.read_csv(population, usecols = ['Country,Other','Population (2020)'])
    '''συνδέουμε το dataframe με τα ελληνικά ονόματα '''
    gr=pd.merge(gr,pop, how='right', left_on='ADMIN', right_on='Country,Other')
    
    '''Ενώνουμε το dataframe του Hopkins και εκείνο του population με βάση την στήλη με τα ελληνικά ονόματα'''
    df=pd.merge(df,gr, how='left', on='ADMIN_GR')

    
    ''' υπολογίζουμε συγκεντρωτικά για κάθε χώρα τα κρούσματα και τους θανάτους ανά ημέρα'''
    df=df.groupby(['Country/Region','Date','ADMIN_GR','Population (2020)'])[columnName].sum().reset_index()
    
    ''' ------------------- ΞΕΚΙΝΑ Η ΟΠΤΙΚΟΠΟΙΗΣΗ ------------------'''
    
    ''' Φτιάχνουμε ένα variable που ΔΕΝ ΘΑ ΠΕΡΙΕΧΕΙ τις χώρες αναφοράς - που θα χρωματίσουμε με άλλο χρώμα'''
    cnt= df[(df['Country/Region']!='Greece') & 
        (df['Country/Region']!='Germany') & 
        (df['Country/Region']!='Italy') & 
        (df['Country/Region']!='United Kingdom') & 
        (df['Country/Region']!='US') & 
        (df['Country/Region']!='Spain')&
        (df['Country/Region']!='China')]
    
    fig = px.line(cnt[(cnt['Country/Region']!='Diamond Princess') & 
                  (cnt['deaths']>=numberCompare)],
              y=columnName,
              color='ADMIN_GR',
              hover_data=['ADMIN_GR'],
              labels={'Date':'Ημερομηνία',
                    columnName:'Αριθμός Κρουσμάτων',
                    'ADMIN_GR':'Χώρα'},
              title=titleGraphic,
              
              line_shape="spline", render_mode="svg",
#               color_discrete_sequence=px.colors.sequential.ice
#               color_discrete_sequence=px.colors.qualitative.Prism
              color_discrete_sequence=['rgb(189,189,189)']
             )
    
    fig.update_layout(paper_bgcolor="#E6ECEC",
                 plot_bgcolor="#E6ECEC",
                 font=dict(
                family="Roboto",
                size=11,
                color="#114B5F")
                 )

    # fig.update_traces(mode="markers+lines", hovertemplate=None)
    # fig.update_layout(hovermode="x")
    fig.update_layout(hovermode="closest",
                     hoverlabel=dict(
                        bgcolor="white", 
                        font_size=12, 
                        font_family="Roboto"),
                     hoverlabel_align = 'left',)

    fig.update_layout(showlegend=True)
    fig.update_layout(legend_title='',

                     legend=dict(
    #                      θέση του legend
    #                     x=0,
    #                     y=1,

    #                      σειρά χωρών reversed, normal, grouped
                        traceorder="reversed", 
                        font=dict(
                            family="roboto",
                            size=10,
                            color="black"
                        ),

                        bgcolor="#E6ECEC",
                        bordercolor="#dadada",
                        borderwidth=.3
                    ))

    fig.update_yaxes(nticks=4,
                    
                    showticklabels=True,
                    showline=True,
                     linewidth=2,
                    linecolor='#114B5F',
                    showgrid=True, 
                     gridwidth=.1, 
                     gridcolor='#F8FAFA',
                    title_text = "Αριθμός κρουσμάτων",
                    title_font = {"size": 11,
                                 'color':'#114B5F'},)

    fig.update_xaxes(tickvals=[5, 10, 15, 20, 25, 30,35,40,45,50,55],
                    ticks=None,
                    showticklabels=True,
                     showline=False,
                     linewidth=.1,
                    linecolor='#F8FAFA',
                    showgrid=True, 
                     gridwidth=.1, 
                     gridcolor='#F8FAFA',
                    title_text = "Ημέρες από το {}o κρούσμα".format(numberCompare),
                    title_font = {"size": 11,
                                 'color':'#114B5F'},
                    )

    fig.update_layout(xaxis_showgrid=True, 
                      yaxis_showgrid=True)


    fig.update_layout(yaxis_type="log")

    fig.update_layout(height=380)

    
    # Γερμανία
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='Germany')&(df[columnName]>numberCompare)][columnName],
                             name='Γερμανία',
                            line=dict(color="black",
                                    width=2),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στην Γερμανία<extra></extra>'
                            ))
    
       # Ιταλία
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='Italy')&(df[columnName]>numberCompare)][columnName],
                             name='Ιταλία',
                            line=dict(color="#3E82B3",
                                    width=2),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στην Ιταλία<extra></extra>'
                            ))

    # UK
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='United Kingdom')&(df[columnName]>numberCompare)][columnName],
                             name='Ηνωμένο Βασίλειο',
                            line=dict(color="#FFD400",
                                    width=2),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στο Ηνωμένο Βασίλειο<extra></extra>'
                            ))


    # USA
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='US')&(df[columnName]>numberCompare)][columnName],
                             name='ΗΠΑ',
                            line=dict(color="lightgreen",
                                    width=2),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στις ΗΠΑ<extra></extra>'
                            ))
    # Spain
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='Spain')&(df[columnName]>numberCompare)][columnName],
                             name='Ισπανία',
                            line=dict(color="purple",
                                    width=2),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στην Ισπανία<extra></extra>'
                            ))
    # Greece
    fig.add_trace(go.Scatter(y=df[(df['Country/Region']=='Greece')&(df[columnName]>numberCompare)][columnName],
                             name='Ελλάδα',
                            line=dict(color="#BA3A0A",
                                    width=3),
                             hovertemplate = '<b>%{y:.f}</b> κρούσματα στην Ελλάδα<extra></extra>'
                            ))


    
    config = dict({'displayModeBar': False, 
                'scrollZoom': False,
                'displaylogo': False,
                'responsive': True,
                'staticPlot': False

                  })
    # μέγεθος γραμμής σε κάθε χώρα στο legend, trace/constant
    fig.update_layout(legend= {'itemsizing': 'constant'},
                     
        annotations = [dict(
            x=-.02,
            y=1.2,
            xref='paper',
            yref='paper',
            text='<i>Λογαριθμική κλίμακα</i>',
            showarrow = False
        )])
    
    fig.show()
    
    return fig.write_json(outputName)
after100Cases(deathsHopkinsFileName, 10, 'deaths', '10_deaths.json','Μετά τους πρώτους 10 θανάτους')

# Heatmap

In [61]:
def heatmap(fileName, columnName, outputName, titleGraphic, keyWord):
    
    ''' φορτώνουμε το πρώτο αρχείο από το Hopkins'''
    df=pd.read_csv(baseHopkinsURL + fileName)\
        .drop(['Lat', 'Long'], axis=1) 
    
    ''' υπολογίζουμε το difference από ημέρα σε ημέρα, ώστε να βγάλουμε τα ΝΕΑ κρούσματα/θάνατοι ανά ημέρα'''
    cols=df.columns.to_list()
    df_dif=df[cols[4:]].diff(axis=1)
    df=df_dif.join(df['Country/Region'])\
        
    ''' αλλάζουμε τα data από wide σε long + μετρέπουμε τη στήλη Date σε datetime'''    
    df=pd.melt(df, id_vars=['Country/Region'], 
                 var_name='Date', value_name=columnName) \
        .astype({'Date':'datetime64[ns]', columnName:'Int64'}, 
                 errors='ignore')
    
    ''' φορτώνουμε το αρχείο με την αντιστοίχιση ελληνικών - αγγλικών ονομάτων χωρών'''
    gr=pd.read_csv(countriesInGreek)
    ''' το συνδέουμε με το dataframe από το Hopkins ώστε να προστεθεί το ADMIN_GR'''
    df=pd.merge(df,gr, how='left', left_on='Country/Region', right_on='ADMIN')
    
    ''' φορτώνουμε το αρχείο με τους πληθυσμούς'''
    pop=pd.read_csv(population, usecols = ['Country,Other','Population (2020)'])
    '''συνδέουμε το dataframe με τα ελληνικά ονόματα '''
    gr=pd.merge(gr,pop, how='right', left_on='ADMIN', right_on='Country,Other')
    
    '''Ενώνουμε το dataframe του Hopkins και εκείνο του population με βάση την στήλη με τα ελληνικά ονόματα'''
    df=pd.merge(df,gr, how='left', on='ADMIN_GR')
    
    ''' υπολογίζουμε συγκεντρωτικά για κάθε χώρα τα κρούσματα και τους θανάτους ανά ημέρα'''
    df=df.groupby(['Country/Region','Date','ADMIN_GR','Population (2020)'])[columnName].sum().reset_index()
    
    
    
    ''' ------------------- ΠΡΟΕΤΟΙΜΑΖΟΥΜΕ ΤΗ ΣΤΗΛΗ ΠΟΥ ΘΑ ΟΠΤΙΚΟΠΟΙΗΣΟΥΜΕ -----------------'''
    
    df[columnName + '_per_hundr']=round((df[columnName]/df['Population (2020)'])*100000,2)
    
    
    
    ''' ------------------- ΕΠΙΛΕΓΟΥΜΕ ΤΙΣ ΧΩΡΕΣ ΜΕ ΠΑΡΟΜΟΙΟ ΠΛΗΘΥΣΜΟ ΜΕ ΤΗΝ ΕΛΛΑΔΑ -----------'''
    
    df=df[(df['Population (2020)']>9000000)\
     & (df['Population (2020)']<12000000)\
     & (df['Date']>'2020-03-06')]
    
    ''' ------------------- ΞΕΚΙΝΑ Η ΟΠΤΙΚΟΠΟΙΗΣΗ ------------------'''
    
    
    countries = df['ADMIN_GR']
    item =df[columnName + '_per_hundr']
    base = datetime.datetime.today()
    dates = df['Date']
    

    fig = go.Figure(data=go.Heatmap(
            z=item,
            x=dates,
            y=countries,
            customdata=df[columnName],

            showscale=True,

            hovertemplate=
            "<b>%{y}</b><br>" +
            "<i>%{x}</i><br><br>" +
            "%{customdata} "+ keyWord +
            "<extra></extra>",

            colorscale= [
            [0, '#E6ECEC'],        #0
            [1./10000, '#dadada'], #10
            [1./1000, '#d7dfe3'],  #100
            [1./100, '#b0bfc7'],   #1000
            [1./10, '#3f6678'],       #10000
            [1., '#BA3A0A']],
            colorbar= dict(
                    title=keyWord + "/<br>100 χιλ ",
                    tick0= 0,
                    tickmode= 'auto', # όταν το tickmode είναι array, τότε παίρνει τα values του tickvals
                    tickvals= [0, 1000,1800])))

    fig.update_layout(title=titleGraphic)

    fig.update_layout(hovermode="closest",
                     hoverlabel=dict(
                        bgcolor="white", 
                        font_size=12, 
                        font_family="Roboto"),
                     hoverlabel_align = 'left',)

    fig.update_layout(paper_bgcolor="#E6ECEC",
                     plot_bgcolor="#E6ECEC",
                     font=dict(
                    family="Roboto",
                    size=11,
                    color="#114B5F")
                     )

    fig.update_layout(height=380, 
                 margin=dict(
                    l=10,
                    r=10,
                    b=20,
                    t=100,
                    pad=1
    ))
    fig.update_xaxes(side="top")

    
    config = dict({'displayModeBar': False, 
                'scrollZoom': False,
                'displaylogo': False,
                'responsive': True,
                'staticPlot': False

                  })
    
    fig.show()
    
    return fig.write_json(outputName)

heatmap(deathsHopkinsFileName,'deaths','death_heat.json','Θάνατοι ανά 100 χιλ. πληθυσμού', 'θάνατοι')

